# Étude comparative des frameworks spécialisés en système de recommandation

Nous étudierons principalement trois frameworks :
* Surprise
* LightFM
* Spotlight

[Surprise](http://surpriselib.com/) est un module spécialisé dans les systèmes de recommandation qui permet d'expérimenter rapidement différents algorithmes très utilisés.

[LightFM](https://github.com/lyst/lightfm) est une implémentation Python des principaux algorithmes utiles aux systèmes de recommandation. Il supporte à la fois des retours implicites et explicites de l'utilisateur. Le papier décrivant l'approche de LightFM est disponible [ici](https://arxiv.org/pdf/1507.08439.pdf).

[Spotlight](https://maciejkula.github.io/spotlight/) a une approche différente et repose sur des réseaux de neurones. Il est essentiellement développé par Maciej Kula, chercheur renommé dans le domaine des systèmes de recommandation. Il utilise la librarie PyTorch.

L'objectif est d'évaluer la qualité des résultats obtenus et les temps d'exécutions. 

## Surprise

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
%time algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
print(accuracy.rmse(predictions))

CPU times: user 2.81 s, sys: 0 ns, total: 2.81 s
Wall time: 2.81 s
RMSE: 0.9414
0.9414018563048242


## Light FM

In [2]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

data = fetch_movielens(min_rating=0.5)

model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)

print("Test precision: {:.2f}".format(precision_at_k(model, data['test'], k=5).mean()))

CPU times: user 2.08 s, sys: 0 ns, total: 2.08 s
Wall time: 1.04 s
Test precision: 0.12


## Spotlight

In [4]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import rmse_score
from spotlight.evaluation import precision_recall_score
from spotlight.factorization.explicit import ExplicitFactorizationModel

dataset = get_movielens_dataset(variant='100K')

train, test = random_train_test_split(dataset)

model = ExplicitFactorizationModel(n_iter=1)
%time model.fit(train)

rmse = rmse_score(model, test)
print(rmse)

precision_at_k, recall_at_k = precision_recall_score(model, test, k=5)
print(precision_at_k.mean())

CPU times: user 3.99 s, sys: 132 ms, total: 4.12 s
Wall time: 587 ms
0.99760884
0.026511134676564158
